In [ ]:
import csv
import re
import os
import pandas as pd
import numpy as np
import xlsxwriter
import shutil

from datetime import datetime, date

In [ ]:
root_dir = '..'
file_root = f'{root_dir}/files'
church_export_root = f'{root_dir}/church'

pricing_breakdown = {
    'student': {
        '05-02-25': 225,
        '05-16-25': 250,
        '05-23-25': 300
    },
    'chaperone': {
        'High School Camp': 150,
        'Middle School Camp': 150,
        'Both Camps': 250,
    }
}

late_fee = 25

church_list = [
    "Ashland City Faith Community",
    "Bent Knee Cowboy",
    "Betel",
    "Bethel (Nashville First)",
    "Bridgepoint",
    "Ciudad de Refugio",
    "Clarksdale Grace Community",
    "Clarksville First",
    "Clarksville Grace",
    "Clarksville Hope Riders Biker",
    "Clarksville Korean Body of Christ",
    "Clarksville Park Lane",
    "Cleveland First",
    "Columbia First",
    "Columbia Grace",
    "Concord Community",
    "Covington",
    "Crossroads (MS)",
    "Crossroads Community",
    "DaySpring Community",
    "Dover First",
    "Eben-Ezer",
    "Elmore",
    "Erin",
    "Fellowship",
    "Fly",
    "Foundry",
    "Franklin Community of Faith",
    "Fulton",
    "Gallatin",
    "Gloster",
    "Gulf Coast Family",
    "Hispana Betel de Gallatin",
    "Gateway Community",
    "Goodlettsville",
    "Gray's Chapel",
    "Griffin's Chapel",
    "Harmony",
    "Hattiesburg First",
    "Hazlehurst",
    "Hendersonville",
    "Hermitage",
    "Hillside Community",
    "Hope Springs",
    "Betel Hermitage",
    "Highland",
    "Jason Chapel",
    "Joelton",
    "Jones Chapel",
    "Kingston Springs",
    "Laurel Bresee",
    "Laurel First",
    "Lewisburg First",
    "Libertad en Cristo",
    "Long Creek",
    "Madison",
    "Madison Hispanic",
    "Magnolia",
    "Memphis Calvary",
    "Memphis Eastside (Hispanic)",
    "Memphis Emmanuel",
    "Memphis Friendship",
    "Memphis Grace",
    "Memphis Holiness Tabernacle",
    "Memphis New Hope",
    "Memphis Refreshing Springs",
    "Meridian Fitkins Memorial",
    "Meridian Northside",
    "Millington",
    "Mt. Wesley-Akin Chapel",
    "Music City West",
    "Nashville Bell Road",
    "Nashville Christ",
    "Nashville Collective",
    "Nashville Community Fellowship",
    "Nashville Donelson",
    "Nashville First (Main Campus)",
    "Nashville First Haitian",
    "Nashville Glencliff Hispanic",
    "Nashville Grace",
    "Nashville Inglewood",
    "Nashville Trevecca Community",
    "Nashville Westwind Community of Faith",
    "New Vision",
    "Nueva Esperanza",
    "Old Hickory",
    "Oxford",
    "Palabra de Vida",
    "Paris First",
    "Pearson Chapel",
    "Pine Hill",
    "Portland",
    "Quitman",
    "Rosebloom",
    "Savannah",
    "Senatobia Nabors Memorial",
    "Spring Hill Restoration Community",
    "Starkville Wesley Community",
    "The House",
    "The Neighborhood Church",
    "Upper Room Fellowship",
    "Victory Praise & Worship",
    "Waverly",
    "Way of the Cross",
    "White House",
    "Woodlawn Community",
    "Individual - No church affiliation",
    "Other"
]

In [ ]:
files = os.listdir(file_root)
files = [f for f in files if os.path.isfile(f'{file_root}/{f}') and f != '.DS_Store'] 
files

In [ ]:
def create_directory(process):
    outdir = f'{root_dir}/{process}/{date.today()}/'
    
    if not os.path.exists(outdir):
        os.mkdir(outdir)

In [ ]:
def write_csv(df, df_name, process, filename):
    create_directory(process)
    df.to_csv(f'{date.today()}_{filename}.csv')

In [ ]:
def write_excel(df, df_name, process, filename, indexed=False):
    create_directory(process)
    df.to_excel(f'{filename}.xlsx', index=indexed, engine='xlsxwriter')

In [ ]:
def move_to_processed(filename):
    src_file = f'{file_root}/{filename}'
    dst_file = f'{root_dir}/processed/{filename}'
    
    shutil.move(src_file, dst_file)

In [ ]:
def strip_filename_from_download_date(filename):
    filename_parts = filename.split('_-_')
    return filename_parts[0]

strip_filename_from_download_date(files[0])

In [ ]:
def get_price(row):
    if row['Are you a student, chaperone, staff'] == 'Chaperone':
        return pricing_breakdown['chaperone'][row['At which camp will you be a chaperone?']]
    
    submission_date_string = row['Date']
    price_dates = pricing_breakdown['student'].keys()
    max_date = datetime.strptime(list(price_dates)[-1], '%m-%d-%y')
    
    for price_date_string in price_dates:
        submission_date = datetime.strptime(submission_date_string, '%b %d, %Y')
        price_date = datetime.strptime(price_date_string, '%m-%d-%y')
        if submission_date <= price_date:
            return pricing_breakdown['student'][price_date_string]
        
        elif submission_date > max_date:
            return pricing_breakdown['student'][list(price_dates)[-1]] + late_fee

In [ ]:
def create_church_info_sheets(file, filename):
    df_original = pd.read_csv(f'{file_root}/{file}')
    df = df_original.copy(deep=True)
    # df['Camp'] = df.apply(lambda x: x['At which camp will you be a chaperone?'] if x['At which camp will you be a chaperone?'] else x['Which Camp?'])
    df['Camp'] = df['At which camp will you be a chaperone?'].combine_first(df['Which Camp?'])

    cols = df_original.columns
    cols = [*cols, 'Camp']
    df = df.loc[:, cols].fillna('')
    df = df.groupby(['What church are you a part of?', 'Last Name', 'First Name'], group_keys=False).apply(lambda x: x)
    
    df.reset_index(inplace=True)
    
    # df['What church are you a part of?'] = df['What church are you a part of?'].apply(lambda x: x.strip())
    for church in church_list:
        df_church = df.loc[df['What church are you a part of?'].str.strip() == church]
        
        if not df_church.empty:
            df_church['Price'] = df_church.apply(lambda x: get_price(x), axis=1)
            df_church['Paid'] = df_church['Payment'].apply(lambda x: int(re.findall("[0-9]+", x)[0]) if re.findall("[0-9]+", x) else 0)
            df_church['Total Due'] = df_church.apply(lambda x: x['Price'] - x['Paid'], axis=1)
            df_church.rename(columns = {"First Name.1": "Guardian First Name", "Last Name.1": "Guardian Last Name","First Name.2": "Youth Leader First Name", "Last Name.2": "Youth Leader Last Name"}, inplace = True)
          
            df_church = df_church.groupby(['Camp'], group_keys=False).apply(lambda x: x)
            df_church.sort_values(['Submission Date', 'Last Name', 'First Name'], inplace=True)
            df_church.reset_index(inplace=True)

            df_church = df_church.drop(columns=['level_0', 'index'])

            df_church.loc['Total',:] = df_church.sum(axis=0, numeric_only=True)

            
            export_path = f"{church_export_root}/{date.today()}/{date.today()}_{church.replace(' ', '_').lower()}"
            write_excel(df_church, church, 'church', export_path)


# create_church_info_sheets(files[0], strip_filename_from_download_date(files[0]))

In [ ]:
df_original

In [ ]:
def build_combined_spreadsheet():
    #TODO: Create New File

    columns_student_worksheet = [
        'Last Name',
        'First Name', #combine to name
        'Church',
        'Shirt Size',
        'Teen/Male',
        'Teen/Female',
        'Adult/Male',
        'Adult/Female',
        'Early Bird',
        'Standard',
        'Late Fee',
        'Adult Fee',
        'Form of Payment/Check #',
        'Amount Owed',
        'Amount Paid',
        'Balance',
    ]

    columns_church_worksheet = [
        'Church Name',
        'Amount Paid',
        'Teen/Male',
        'Teen/Female',
        'Adult/Male',
        'Adult/Female',
        'Amount Balance',
        'Amount Paid',
        'Totals',
        'Small',
        'Medium',
        'Large',
        'XL',
        '2X',
        '3X',
        '4X',
        '5X',
    ]

    create_directory(process)
    df.to_excel(f'{filename}.xlsx', index=indexed, engine='xlsxwriter')

In [ ]:
for file in files:
    print(file)
    filename = strip_filename_from_download_date(file)
    print(filename)
    create_church_info_sheets(file=file, filename=filename)
    # build_combined_spreadsheet()
    move_to_processed(file)